In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [1]:
# Set your own project id here
PROJECT_ID = 'kmiki-kaggle'
from google.cloud import bigquery
bigquery_client = bigquery.Client(project=PROJECT_ID)
#from google.cloud import storage
#storage_client = storage.Client(project=PROJECT_ID)
#from google.cloud import automl_v1beta1 as automl
#automl_client = automl.AutoMlClient()

トレーニングデータ確認

In [1]:

%%bigquery --project $PROJECT_ID
SELECT
*
FROM
`kaggleworkshops.google_analytics_sample.train`
LIMIT 5

テストデータ確認

In [1]:
%%bigquery --project $PROJECT_ID
SELECT
*
FROM
`kaggleworkshops.google_analytics_sample.test`
LIMIT 5

In [1]:
%%bigquery --project $PROJECT_ID
CREATE OR REPLACE MODEL bqmlretail.logreg_sample
TRANSFORM(
  LOWER(country) country_lower,
  totalPageviews,
  addedToCart
)
OPTIONS(
  model_type='logistic_reg', 
  INPUT_LABEL_COLS=["addedToCart"]) AS
SELECT 
  country,
  totalPageViews,
  addedToCart
FROM
  `kaggleworkshops.google_analytics_sample.train`
WHERE date BETWEEN TIMESTAMP("2016-08-01")
  AND TIMESTAMP("2016-08-31")

モデルの非同期トレーニング

In [1]:
sqlquery = """
CREATE OR REPLACE MODEL bqmlretail.logreg_sample
TRANSFORM(
  LOWER(country) country_lower,
  totalPageviews,
  addedToCart
)
OPTIONS(
  model_type='logistic_reg', 
  INPUT_LABEL_COLS=["addedToCart"]) AS
SELECT 
  country,
  totalPageViews,
  addedToCart
FROM
  `kaggleworkshops.google_analytics_sample.train`
WHERE date BETWEEN TIMESTAMP("2016-08-01")
  AND TIMESTAMP("2016-08-31")
"""

job_config = bigquery.QueryJobConfig()
bigquery_client.query(sqlquery, job_config=job_config)  # Make an API request. 

トレーニングが完了しているか確認

In [1]:
#List models
dataset_id = "bqmlretail"
models = bigquery_client.list_models(dataset_id)
print("Listing Trained BigQuery ML Models:")
print(["{}.{}".format(model.dataset_id, model.model_id) for model in models])

ML.PREDICTによる予測

In [1]:
%%bigquery pred --project $PROJECT_ID

SELECT
  sessionId, 
  prob as addedToCart
FROM
  ML.PREDICT(MODEL bqmlretail.logreg_sample, ## replace model name
    (
    SELECT 
      CONCAT(fullVisitorId, CAST(visitStartTime as string)) as sessionId, 
      *
    FROM
    `kaggleworkshops.google_analytics_sample.test`
  )),
UNNEST(predicted_addedToCart_probs)
WHERE label = 1

In [1]:
## Check results in dataframe
pred.head()

提出ファイル生成

In [1]:
## Save predictions as CSV
pred.to_csv("logregsample.csv", index=False)
!ls

ノートブック右上の[Data]を開き、CSVファイルにカーソルを合わせ、3つのドットをクリックして、ファイルをコンピューターにダウンロードします。

次に、[送信]ページから予測を送信します。